In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression

### 50 Feature Selection Analysis wiht Sentiment for EIHOTEL.BO

In [2]:
df = pd.read_csv('../../../datasets/processed_data/combined_features/EIHOTEL.BO.csv')

In [3]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'high', 'low',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change',
       'yesterday_close', 'ln_target'],
      dtype='object', length=635)

In [4]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [5]:
df = create_custom_target(df)

In [6]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [7]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [8]:
df_without_date = df.drop(columns=["date"])

In [9]:
df_without_date.dropna(inplace=True)

In [10]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [11]:
window_size = 10

In [12]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# Identify the indices of the specified sentiment columns in the original dataset
sentiment_columns = ['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound']
column_indices = [train_data.columns.get_loc(col) for col in sentiment_columns if col in train_data.columns]

column_indices

[616, 617, 618, 619, 620, 621]

In [14]:
# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [15]:
# Use SelectKBest to select the top 44 features
selector = SelectKBest(score_func=f_regression, k=44)
X_train_44 = selector.fit_transform(X_train, y_train)
X_test_44 = selector.transform(X_test)

# Determine if the specified columns are among the top 44 features
selected_indices = selector.get_support(indices=True)
missing_indices = [idx for idx in column_indices if idx not in selected_indices]

# Add the specified columns if they are not among the top 44 features
for idx in missing_indices:
    extracted_column_train = X_train.iloc[:, idx].values.reshape(-1, 1)
    extracted_column_test = X_test.iloc[:, idx].values.reshape(-1, 1)
    X_train_44 = np.hstack([X_train_44, extracted_column_train])
    X_test_44 = np.hstack([X_test_44, extracted_column_test])

# If some of the specified columns were already in the top 44, 
# select additional top features to make the total count 50
remaining_indices = [i for i in range(X_train.shape[1]) if i not in selected_indices and i not in missing_indices]
num_additional_features_needed = 50 - X_train_44.shape[1]

if num_additional_features_needed > 0:
    additional_selector = SelectKBest(score_func=f_regression, k=num_additional_features_needed)
    additional_selector.fit(X_train.iloc[:, remaining_indices], y_train)
    X_train_additional = additional_selector.transform(X_train.iloc[:, remaining_indices])
    X_test_additional = additional_selector.transform(X_test.iloc[:, remaining_indices])

    X_train_44 = np.hstack([X_train_44, X_train_additional])
    X_test_44 = np.hstack([X_test_44, X_test_additional])

X_train_50 = X_train_44
X_test_50 = X_test_44

In [16]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [17]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [18]:
def evaluate_model(model, window, test_data, test_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(test_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(test_data, window, y_pred)

    test_dates = test_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': test_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [19]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [20]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23   74.900002   74.176597
 1    2020-06-24   74.949997   77.801178
 2    2020-06-25   68.900002   72.418431
 3    2020-06-26   69.000000   69.634361
 4    2020-06-29   68.750000   68.532306
 ..          ...         ...         ...
 722  2023-05-24  205.600006  205.609136
 723  2023-05-25  215.850006  205.381502
 724  2023-05-26  207.699997  211.003559
 725  2023-05-30  207.850006  210.421153
 726  2023-05-31  210.750000  213.090157
 
 [727 rows x 3 columns],
 0.021121700768089183,
 3.692423256835495)

In [21]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 2.11%


In [22]:
predictions_df.to_csv("rf_EIHOTEL.csv", index=False)

In [23]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df_copy = predictions_df.copy(deep=True)


predictions_df_copy['date'] = pd.to_datetime(predictions_df_copy['date'])

predictions_df_copy['label'] = 'Actual'
predictions_df_copy['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df_copy).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df_copy).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

In [24]:
feature_names = df.columns.tolist()

# Get feature importances
importances = model_50.feature_importances_

# Map importances to corresponding feature names
feature_importance_mapping = dict(zip(feature_names, importances))

# Sort by importance
sorted_features = sorted(feature_importance_mapping.items(), key=lambda x: x[1], reverse=True)

# Display top 10
for feature, importance in sorted_features[:50]:
    print(f"{feature}: {importance}")


volatility_dcp: 0.06088296648936049
volatility_dcw: 0.058952495492758675
trend_vortex_ind_pos: 0.05575246215682179
volatility_kcli: 0.05234346064560937
trend_macd: 0.04768432756567811
trend_ema_slow: 0.045549394123826345
volatility_bbp: 0.04484498857219208
volume_vpt: 0.04444826109653783
volatility_atr: 0.04197292270839926
date: 0.04018222590058402
volume_fi: 0.03997100457133587
adj close: 0.0373713352322438
volatility_dch: 0.0372751227274908
volatility_bbhi: 0.035187828023016704
volatility_dcl: 0.034958256100944864
open: 0.03453337432225379
volatility_kcc: 0.033969137391137896
trend_vortex_ind_neg: 0.02560751082660528
trend_vortex_ind_diff: 0.022332480384741796
volume_em: 0.022022597031722074
volume_mfi: 0.00878998588266218
volume_obv: 0.008297297320872428
close: 0.007928393105042755
high: 0.007895229270149932
volume: 0.007774632699395653
volume_adi: 0.007581904144160645
low: 0.007512256344093863
trend_macd_signal: 0.007276039497176443
trend_ema_fast: 0.007218937897649782
volatility_d

### 50 Feature Selection Analysis wihtout Sentiment for EIHOTEL.BO

In [26]:
df = pd.read_csv('../../../datasets/processed_data/combined_features/EIHOTEL.BO.csv')

In [27]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [50]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [51]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [52]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

# commented by Amit Jha
# dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [53]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [54]:
df_without_date.dropna(inplace=True)

In [55]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [56]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [57]:
# Train the model on the top 50 features
model_50 = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    bootstrap=True
)
model_50.fit(X_train_50, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=200)

In [58]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [64]:
predictions_df

,date,y_test,y_pred,label,predicted_label
0,2020-06-23,74.900002,73.934482,Actual,Predicted
1,2020-06-24,74.949997,77.604654,Actual,Predicted
2,2020-06-25,68.900002,72.372198,Actual,Predicted
3,2020-06-26,69.000000,69.635824,Actual,Predicted
4,2020-06-29,68.750000,68.398571,Actual,Predicted
...,...,...,...,...,...
722,2023-05-24,205.600006,205.284501,Actual,Predicted
723,2023-05-25,215.850006,205.073905,Actual,Predicted
724,2023-05-26,207.699997,210.383420,Actual,Predicted
725,2023-05-30,207.850006,209.799243,Actual,Predicted


In [43]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2021-12-14   74.900002   74.025669
 1    2021-12-15   74.949997   77.597664
 2    2021-12-16   68.900002   72.429923
 3    2021-12-17   69.000000   69.744987
 4    2021-12-20   68.750000   68.539190
 ..          ...         ...         ...
 722         NaN  205.600006  204.974315
 723         NaN  215.850006  204.864443
 724         NaN  207.699997  210.560422
 725         NaN  207.850006  209.701173
 726         NaN  210.750000  212.169486
 
 [727 rows x 3 columns],
 0.01911791870142192,
 3.5099767467977387)

In [59]:
predictions_df.to_csv("rf_EIHOTEL_without_senitment.csv", index=False)

In [60]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 2.02%


In [61]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

In [63]:
feature_names = df.columns.tolist()

# Get feature importances
importances = model_50.feature_importances_

# Map importances to corresponding feature names
feature_importance_mapping = dict(zip(feature_names, importances))

# Sort by importance
sorted_features = sorted(feature_importance_mapping.items(), key=lambda x: x[1], reverse=True)

# Display top 10
for feature, importance in sorted_features[:10]:
    print(f"{feature}: {importance}")

trend_sma_fast: 0.06790550342316608
volatility_atr: 0.06160972835838349
trend_macd_diff: 0.06065152226004214
trend_ema_slow: 0.058553265624672945
volatility_kcw: 0.053655087243568836
trend_sma_slow: 0.05317343411832921
volume_em: 0.05125368065241866
volatility_bbl: 0.04994099255915349
date: 0.04828460375469958
volatility_kcli: 0.04259621291049493


### An explaination of key features:
volume_vwap: Volume Weighted Average Price (VWAP) - an average price based on both volume and price. Commonly used as a benchmark.

volume_vpt: Volume Price Trend - it indicates the amount of volume, which can be attributed to the trend's direction.

date: Represents the specific date for each data entry.

adj close: Adjusted closing price accounts for events such as dividends, stock splits, and new stock offerings.

volume_adi: Accumulation/Distribution Index - relates volume to price change.

high: The highest price of the stock during a particular period.

volatility_bbh: Bollinger Bands High - a volatility indicator, representing the upper band.

volume_sma_em: Volume Simple Moving Average Exponential - the exponential moving average of volume.

volatility_dcw: Donchian Channel Width - shows the size of the Donchian Channel, indicating volatility.

volume_obv: On-Balance Volume - uses volume flow to predict price changes.

volume_em: Volume Exponential Moving - the exponential moving average of volume.

volume: The number of shares or contracts traded in a stock or an entire market during a given period.

volume_fi: Force Index - measures the pressure of either buying or selling.

open: The price at which a stock started trading during a particular period.

trend_mass_index: Mass Index - identifies reversals when the range widens or narrows.

volatility_bbm: Bollinger Bands Middle - the middle band of the Bollinger Bands.

volatility_ui: Ulcer Index - measures downside risk.

volume_cmf: Chaikin Money Flow - measures the amount of Money Flow Volume over a certain period.

volatility_kcw: Keltner Channel Width - shows the width between Keltner Channels, indicating volatility.

volatility_kcc: Keltner Channel Central - the middle line of the Keltner Channel.

volume_nvi: Negative Volume Index - focuses on periods where volume has decreased from the previous period.

volatility_bbli: Bollinger Bands Low Indicator - a volatility indicator, representing the lower band.

trend_vortex_ind_diff: Vortex Indicator Difference - shows the difference between VI+ and VI-.

volatility_bbw: Bollinger Bandwidth - shows the width between the Bollinger Bands.

trend_trix: TRIX - shows the percent rate of change of a triple exponentially smoothed moving average.

trend_ema_slow: Exponential Moving Average Slow - places a greater weight and significance on the most recent data points.

trend_vortex_ind_pos: Vortex Indicator Positive - an indicator used for spotting trend reversals.

volatility_kcl: Keltner Channel Low - the lower band of the Keltner Channel.

volatility_atr: Average True Range - measures stock volatility.

volatility_bbp: Bollinger Bands Percentage - relates the current price to the Bollinger Bands levels.

volatility_kch: Keltner Channel High - the upper band of the Keltner Channel.

volatility_kcli: Keltner Channel Low Indicator - the lower channel line of the Keltner Channel.

volatility_dcl: Donchian Channel Low - the lowest price of the last 'n' days.

volatility_kchi: Keltner Channel High Indicator - the upper channel line of the Keltner Channel.

volatility_bbhi: Bollinger Bands High Indicator - indicates if the price is above the upper Bollinger band.

trend_macd: Moving Average Convergence Divergence - shows the relationship between two moving averages of a stock’s price.

trend_sma_fast: Simple Moving Average Fast - the average stock price over a specific period, giving equal weight to every daily price.

trend_sma_slow: Simple Moving Average Slow - similar to SMA fast but considers a longer period.

trend_ema_fast: Exponential Moving Average Fast - like SMA but gives more weight to recent prices.

volatility_dcm: Donchian Channel Middle - the average of the Donchian high and low.

volatility_dch: Donchian Channel High - the highest price of the last 'n' days.

trend_macd_diff: The difference between MACD and its signal line.

trend_macd_signal: The signal line of the MACD, which is an EMA of the MACD.

low: The lowest price of the stock during a particular period.

close: The final price at which the stock is traded during a particular period.